In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setMaster("yarn").setAppName("Lab4")
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.cores.max", "2")
sc = SparkContext(conf=conf)
sc.getConf().getAll()

[('spark.ui.proxyBase', '/proxy/application_1638081630762_0004'),
 ('spark.app.name', 'Lab4'),
 ('spark.driver.memory', '4g'),
 ('spark.executorEnv.PYTHONPATH',
  '/usr/local/spark/python:<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.9-src.zip'),
 ('spark.executor.memory', '4g'),
 ('spark.driver.port', '33559'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.bindAddress', '192.1.1.11'),
 ('spark.app.startTime', '1638090766293'),
 ('spark.ui.port', '4050'),
 ('spark.master', 'yarn'),
 ('spark.driver.appUIAddress', 'http://edge:4050'),
 ('spark.rdd.compress', 'True'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://yarnmaster:8088/proxy/application_1638081630762_0004'),
 ('spark.yarn.jars', ''),
 ('spark.driver.host', 'edge'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.cores.max', '2'),
 ('spark.app.id', 'application_1638081630762_0004'),
 ('spark.submit.pyFiles', ''),
 ('spark.yarn.isPython', 'true'),
 ('spark.submi

In [2]:
rawRDD = (sc.textFile("hdfs://yarnmaster:9000/itemsets_extra",16))
itemsetRDD = rawRDD.map(lambda x: sorted([int(item) for item in x.strip().split(' ')]))
freqItemset = itemsetRDD.flatMap(lambda x: set(x)).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b) \
.filter(lambda x: x[1]>99).map(lambda x: x[0])

###### QUESTION 1: Find the products which are frequently browsed together by using the A-priori algorithm.

freqItemset : candidate for k = 1

In [3]:
freqSet = set(freqItemset.collect())
def getFreqItemOnly(x):
    res = []
    for it in x:
        if it in freqSet:
            res.append(it)
    return res

freqRDD = itemsetRDD.map(getFreqItemOnly).filter(lambda x: len(x)>1)

+ Convert freqItemset to set for faster element check (I saw the in implementation of paper they even used tree structure)
+ Filter out item that not in candidate k=1 and basket that not contains more than 1 item

In [4]:
%%time
import itertools
pairRDD = freqRDD.flatMap(lambda x: itertools.combinations(x,2))
pairCandidate = pairRDD.map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b) \
.filter(lambda x: x[1]>99)
pairCandidate.take(5)

CPU times: user 21.6 ms, sys: 0 ns, total: 21.6 ms
Wall time: 56 s


[((21, 401), 2070),
 ((21, 529), 2521),
 ((35, 983), 574),
 ((54, 460), 1392),
 ((175, 883), 968)]

+ Use itertools to generate combination of candidate pairs in each basket
+ simply count and filter out pair that not match support thresh

In [5]:
freqPairItemSet = pairCandidate.flatMap(lambda x: [x[0][0],x[0][1]]).distinct().collect()

freqPairItemSet: items that appear in frequent pairs candidate

.........................

In [6]:
def getFreqItemFromPairOnly(x):
    res = []
    for it in x:
        if it in freqPairItemSet:
            res.append(it)
    return res


freqPairRDD = freqRDD.map(getFreqItemFromPairOnly).filter(lambda x: len(x)>2)

filter out items that not appear in frequent pairs candidate

In [7]:
%%time
tripleRDD = freqPairRDD.flatMap(lambda x: itertools.combinations(x,3))
tripleCandidate = tripleRDD.map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b) \
.filter(lambda x: x[1]>99)
tripleCandidate.take(5)

CPU times: user 245 ms, sys: 48.9 ms, total: 294 ms
Wall time: 57min 4s


[((518, 828, 947), 165),
 ((694, 775, 790), 103),
 ((272, 510, 579), 645),
 ((368, 401, 794), 148),
 ((48, 438, 795), 1185)]

+ generate combinations of 3 in each basket
+ count and filter

# Question 2: 

In [8]:
def genTripletPair(x):
    res = []
    res.append(((x[0][0],x[0][1]),(x[0][2],x[1])))
    res.append(((x[0][0],x[0][2]),(x[0][1],x[1])))
    res.append(((x[0][1],x[0][2]),(x[0][0],x[1])))
    return res

tripleAsso = tripleCandidate.flatMap(genTripletPair)

+ ((x,y,z), value) -> ((x,y),(z,value)), ((x,z),(y,value)), ((y,z),(x,value)) => prepare key for join with pairCandidate
+ Only need 3 because x<y<z (sorted at the beginning)

In [9]:
%%time
def genRule(x):
    return (str(x[0])+ " -> " + str(x[1][0][0]), round(x[1][0][1]/x[1][1],2))
            
tripleAsso.join(pairCandidate).map(genRule).filter(lambda x:x[1]>0.7).sortBy(lambda x: x[1], ascending=False).take(5)

CPU times: user 103 ms, sys: 8.25 ms, total: 111 ms
Wall time: 13min 13s


[('(259, 848) -> 411', 1.0),
 ('(289, 586) -> 766', 1.0),
 ('(717, 766) -> 554', 1.0),
 ('(529, 880) -> 145', 1.0),
 ('(781, 916) -> 900', 1.0)]

+ Join with pair candidates
+ do the division
+ filter out rules that has conf <0.7
+ sorting
+ get top 5